# Assignment 6

### Part A

In [3]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [4]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [9]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    
    er = []
    
    for i in range(len(data)) :
        row = data.iloc[i]
        er.append((row['Price in 1 Year']/P - 1) * row['Probability'])
    
    return np.sum(er)

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [13]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    
    er = exp_return(data, P)
    vol = []
    
    for i in range(len(data)) :
        row = data.iloc[i]
        pr = row['Price in 1 Year']/P - 1
        vol.append(row['Probability'] * (pr - er)**2)
    
    return np.sqrt(np.sum(vol))

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [15]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    
    z_score = 1.96
    moe = z_score * volatility(data, P) / np.sqrt(len(data))
    er = exp_return(data, P)
    
    return [er - moe, er + moe]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [17]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    
    rp = market_return - rf
    er = rf + beta * rp
    
    return er

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [25]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    
    data = pd.read_csv(path)
    data = data.dropna()
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')
    data = data.dropna()
    data = data.groupby('PERMNO').filter(lambda x: len(x) >= 500)
    data.reset_index(inplace = True)
    data = data.drop('index', axis=1)
    data = data.dropna()
    
    return data

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [124]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Average Return" : []
    })
    CUSIP = data['CUSIP'].drop_duplicates()
    ret = data.groupby('CUSIP')['RET']
    avg_ret = list(ret.mean())
    
    output['CUSIP'] = CUSIP
    output['Average Return'] = avg_ret
    output.reset_index(inplace=True)
    output = output.drop('index', axis=1)
    
    return output

In [152]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    
    output = pd.DataFrame(data = {
        "CUSIP" : [],
        "Volatility" : []
    })

    CUSIP = data['CUSIP'].drop_duplicates()
    ret = data.groupby('CUSIP')['RET']
    vol = list(ret.var())
    
    output['CUSIP'] = CUSIP
    output['Volatility'] = np.sqrt(vol)
    output.reset_index(inplace=True)
    output = output.drop('index', axis=1)
        
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [156]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    
    corr = ret_data['Average Return'].corr(vol_data['Volatility'])
    
    return corr